In [0]:
%sql
-- Set the environment
USE CATALOG workspace;
CREATE SCHEMA IF NOT EXISTS sql_analytics_lab;
USE SCHEMA sql_analytics_lab;

-- Create Mock Sales Data
CREATE OR REPLACE TABLE sales_data AS
SELECT 
    id as order_id,
    date_add(current_date(), CAST(-(id % 30) AS INT)) as order_date,
    (id * 123 % 500) as customer_id,
    CASE 
        WHEN id % 3 = 0 THEN 'Electronics'
        WHEN id % 3 = 1 THEN 'Fashion'
        ELSE 'Home & Garden'
    END as category,
    (id * 77 % 1000) as sale_amount,
    CASE 
        WHEN id % 10 < 7 THEN 'Completed'
        WHEN id % 10 < 9 THEN 'Pending'
        ELSE 'Cancelled'
    END as status
FROM range(1, 501);

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Revenue Trends (Daily Aggregation)
SELECT 
    order_date, 
    SUM(sale_amount) as daily_revenue,
    AVG(SUM(sale_amount)) OVER(ORDER BY order_date ROWS BETWEEN 7 PRECEDING AND CURRENT ROW) as moving_avg_7d
FROM sales_data
WHERE status = 'Completed'
GROUP BY order_date
ORDER BY order_date;

order_date,daily_revenue,moving_avg_7d
2025-12-24,7232,7232.0
2025-12-25,8000,7616.0
2025-12-26,7768,7666.666666666667
2025-12-27,7536,7634.0
2025-12-28,7304,7568.0
2025-12-29,8072,7652.0
2025-12-30,8340,7750.285714285715
2026-01-03,8104,7794.5
2026-01-04,7795,7864.875
2026-01-05,7486,7800.625


In [0]:
%sql
-- Top Performing Categories (Window Function)
SELECT 
    category,
    SUM(sale_amount) as total_revenue,
    RANK() OVER (ORDER BY SUM(sale_amount) DESC) as category_rank
FROM sales_data
GROUP BY category;

category,total_revenue,category_rank
Electronics,79891,1
Fashion,79750,2
Home & Garden,79609,3


In [0]:
%sql
-- Sales Funnel (Conversion Logic)
WITH funnel_base AS (
    SELECT 
        count(distinct customer_id) as total_users,
        count(distinct CASE WHEN status != 'Cancelled' THEN customer_id END) as engaged_users,
        count(distinct CASE WHEN status = 'Completed' THEN customer_id END) as converted_users
    FROM sales_data
)
SELECT 'Total Users' as stage, total_users as count FROM funnel_base
UNION ALL
SELECT 'Engaged Users', engaged_users FROM funnel_base
UNION ALL
SELECT 'Converted Users', converted_users FROM funnel_base;

stage,count
Total Users,500
Engaged Users,450
Converted Users,350
